In [117]:
import os, sys
from PIL import Image
import glob
from dotenv import load_dotenv
import numpy as np
from matplotlib import pyplot as plt
import openai
import logging as log
import time
from io import BytesIO

import json
import base64

load_dotenv()

True

In [118]:
MAX_RES = 1024

In [121]:
def create_alpha_padding_for_image(image, type):
    w, h = image.size
    match type:
        case "top" | "bottom": #horizontal padding
            size = (w, MAX_RES//2)
        case "left" | "right": #vertical padding
            size = (MAX_RES//2, h)
    
    padding = Image.new(mode="RGBA", size=size, color=(0, 0, 0, 0))
    return padding

In [122]:
def merge_images(image1, image2, type="bottom"):

    w1, h1 = image1.size
    w2, h2 = image2.size

    match type:
        case "top" | "bottom":
            new_size = (max(w1, w2) , h1 + h2)
        case "left" | "right":
            new_size = (w1 + w2 , max(h1, h2))

    merged = Image.new("RGBA", new_size)

    match type:
        case "top":
            merged.paste(image2)
            merged.paste(image1, (0, h2))
        case "bottom":
            merged.paste(image1)
            merged.paste(image2, (0, h1))
        case "right":
            merged.paste(image1)
            merged.paste(image2, (w1, 0))
        case "left":
            merged.paste(image2)
            merged.paste(image1, (w2, 0))
    
    return merged

In [123]:
def add_padding(img_path, type="top"):

    img = Image.open(img_path)

    padding = create_alpha_padding_for_image(img, type)
    merged = merge_images(img, padding, type)
    merged.save(f"test.png")


In [124]:
def get_box_selection(image, left, top):
    
    box = (left, top, left+MAX_RES, top+MAX_RES)

    #check if box is not out of image bounds
    w, h = image.size

    if box[2] > w:
        box[2] = w

    if box[3] > h:
        box[3] = h

    return box



In [125]:
def parse_image(image_data):

    image = Image.open(BytesIO(base64.b64decode(image_data)))
    return image

In [126]:
def image_to_byte_array(image):
    byte_stream = BytesIO()
    image.save(byte_stream, format='PNG')
    byte_array = byte_stream.getvalue()
    return byte_array

In [127]:
def openai_edit_image(image, text_prompt, n=4, format="b64_json"):

    try:
        log.info("Generating images...")
        start_t = time.perf_counter()

        response = openai.Image.create_edit(
            image=image,
            prompt=text_prompt,
            n=n,
            size="1024x1024",
            response_format=format,
        )

        duration = time.perf_counter() - start_t
        log.info(f"Generating images took {duration:.2f}s")

        return response, "Images generated successfully!"

    except openai.error.OpenAIError as e:
        log.info(e.http_status)
        log.info(e.error)
        log.info(e)
        return None, str(e)

In [128]:
def modify_image(img_path, prompt, left, top):
    img = Image.open(img_path)
    box = get_box_selection(img, left, top)
    crop = img.crop(box)
    crop_bytes = image_to_byte_array(crop)
    r, msg = openai_edit_image(crop_bytes, prompt, n=10)

    print(msg)

    for i, data in enumerate(r['data']):
        variant = parse_image(data["b64_json"])
        img.paste(variant, box)
        img.save(f"{i+1}-{prompt}.png")    

In [131]:
img_path = to_extend[0]
add_padding(img_path, "bottom")
modify_image("test.png", "dalmatian dog", 0, MAX_RES//2)

Images generated successfully!
